In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json

In [15]:
ext=('.json')
attackerlist=[]
for files in os.listdir():
    if files.__contains__('A2'):
        attackerlist.append(int(files.split("-")[2]))
print(attackerlist)

[145, 1993, 1369, 853, 2965, 121, 1729, 1765, 343, 973, 2671, 223, 649, 1513, 2029, 517, 631, 2377, 2653, 7, 511, 313, 385, 1339, 2557, 1081, 2851, 2701, 2239, 2827, 2533, 1381, 2713, 595, 1309, 1711, 1429, 703, 1873, 2683, 1627, 1351, 1069, 1453, 1759, 1201, 1663, 2269, 1375, 1165, 1939, 1903, 2329, 2347, 541, 493, 2743, 2563, 2905, 103, 2107, 2215, 2785, 1597, 1189, 2125, 2809, 1645, 2755, 2575, 1141, 2353, 361, 373, 2287, 1231, 1387, 1621, 2005, 2491, 679, 607, 1117, 2185, 1699, 913, 2509, 1333, 2857, 1405, 907, 2761, 2233, 895, 2437, 1885, 181, 2041, 481, 799, 733, 1657, 97, 2815, 1831, 2245, 229, 589, 643, 1753, 349, 1009, 763, 2419, 1615, 1105, 667]


In [20]:
for files in os.listdir():
    if files.endswith(ext):
        atk = pd.read_json(files,lines=True)
        atk.drop(['noise','spd_noise','pos_noise'], inplace=True, axis=1,errors='ignore')
        atk['reciever']=int(files.split("-")[2])
        atk1 = pd.DataFrame(atk['pos'].tolist(), columns=['pos-x1', 'pos-y1', 'pos-z'])
        atk1.drop(['pos-z'], inplace=True, axis=1,errors='ignore')
        atk2 = pd.DataFrame(atk['spd'].tolist(), columns=['spd-x1', 'spd-y1', 'spd-z'])
        atk2.drop(['spd-z'], inplace=True, axis=1,errors='ignore')
        frames1=[atk,atk1,atk2]
        attack=pd.concat(frames1,axis=1)
        attack.drop(['pos','spd'], inplace=True, axis=1,errors='ignore')
        bsm = attack[attack["type"] == 3]
        cs = ['pos-x1','pos-y1','spd-x1','spd-y1']
        g= bsm[cs].shift(-1).add_prefix('rec')
        bsm = pd.concat([bsm, g], axis=1).fillna(0).astype(float)
        bsm['attacker']=0
        bsm.loc[bsm["sender"].isin(attackerlist), "attacker"] = 1
        bsm.drop(['type','rcvTime','sendTime','messageID'], inplace=True, axis=1,errors='ignore')
        bsm.to_csv('bsm.csv', mode='a', header=False, index=False)


KeyError: 'sender'

In [17]:
bsm

,type,rcvTime,reciever,pos-x1,pos-y1,spd-x1,spd-y1,recpos-x1,recpos-y1,recspd-x1,recspd-y1,attacker


In [21]:
col_names = ['sender','RSSI','sen-px','sen-py','sen-sx','sen-sy','reciever','rec-px','rec-py','rec-sx','rec-sy','attacker']
# load dataset
pi = pd.read_csv("bsm.csv", header=None, names=col_names)
pi['del-px'] = pi['sen-px'] - pi['rec-px']
pi['del-py'] = pi['sen-py'] - pi['rec-py']
pi['del-sx'] = pi['sen-sx'] - pi['rec-sx']
pi['del-sy'] = pi['sen-sy'] - pi['rec-sy']
pi['attack']= pi['attacker']
pi.drop(['attacker'], inplace=True, axis=1)
pi

,sender,RSSI,sen-px,sen-py,sen-sx,sen-sy,reciever,rec-px,rec-py,rec-sx,rec-sy,del-px,del-py,del-sx,del-sy,attack
0,181.0,5.604203e-09,1021.0,6047.559677,5050.303481,12.719135,-6.328671,5502.208645,5409.754442,6.277599,-10.483078,-4481.208645,637.805235,5044.025882,23.202213,1
1,559.0,1.490972e-08,1021.0,5502.208645,5409.754442,6.277599,-10.483078,6060.254145,5043.987084,12.658486,-6.298493,-5039.254145,458.221561,5397.095956,12.576092,0
2,181.0,3.347665e-09,1021.0,6060.254145,5043.987084,12.658486,-6.298493,5310.504604,5546.929339,4.956657,-9.202118,-4289.504604,513.324806,5039.030427,21.860603,1
3,469.0,4.366793e-09,1021.0,5310.504604,5546.929339,4.956657,-9.202118,5506.276383,5398.373940,2.645954,-11.910720,-4485.276383,-87.869336,5544.283384,16.867376,0
4,559.0,8.630784e-08,1021.0,5506.276383,5398.373940,2.645954,-11.910720,5307.270529,5556.782258,-10.145284,-4.107305,-4286.270529,-50.505875,5408.519223,6.753259,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415031,2899.0,8.195576e-08,2815.0,4538.924695,5212.945520,-11.071124,7.654111,4457.794819,5286.178225,10.091648,-6.943091,-1642.794819,-747.253530,5202.853872,-4.128034,0
415032,1063.0,1.970611e-08,2815.0,4457.794819,5286.178225,10.091648,-6.943091,4435.711015,5301.694813,10.983071,-5.311970,-1620.711015,-843.899994,5275.195154,15.403618,0
415033,997.0,9.294125e-09,2815.0,4435.711015,5301.694813,10.983071,-5.311970,4761.647239,5090.866458,-9.677150,7.110421,-1946.647239,-655.155443,5311.371963,3.872650,0
415034,2785.0,1.421286e-06,2815.0,4761.647239,5090.866458,-9.677150,7.110421,4515.445967,5233.983299,-10.488608,7.695976,-1700.445967,-472.336060,5101.355066,-17.373126,1


In [22]:
X = pi.iloc[:, :-1].values
y = pi.iloc[:, 15].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/Users/adithyabalaji/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[[305953  12501]
 [ 36270  56162]]
              precision    recall  f1-score   support

           0       0.89      0.96      0.93    318454
           1       0.82      0.61      0.70     92432

    accuracy                           0.88    410886
   macro avg       0.86      0.78      0.81    410886
weighted avg       0.88      0.88      0.87    410886

